Вариант 7 Глазунов Даниил ПИ19-4

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
import sympy as sym
import matplotlib.pyplot as plt
from gurobipy import *

In [2]:
a = 3
b = 2
c = 23

In [3]:
#Находим оптимальный план производства
#Вводим границы и ограничения
bounds = np.array([(0, np.inf), (0, np.inf), (0, np.inf)])

#A_ma - матрица при x1,x2,x3
A_ma = np.array([[1, 3, a], [6, 5, 2]])
#x <= B_ma
B_ma = np.array([3000, 3320])
#c0 - коэффициенты перед переменными в целевой функции
c0 = np.array([6*b + 12, 5*b + 22, c])

In [4]:
res = linprog(-c0, A_ma, B_ma, bounds=bounds, method='simplex', options={'disp': True})

Optimization terminated successfully.
         Current function value: -27042.500000
         Iterations: 3


In [5]:
x1 = res.x[0]
x2 = res.x[1]
x3 = res.x[2]
F_max = -res.fun
print(res.x)

[247.5   0.  917.5]


In [6]:
print(f'Оптимальные значения: x1 = {x1.round(3)}, x2 = {x2.round(3)}, x3 = {x3.round(3)}')
print('Max значение функции:',F_max.round(3))

Оптимальные значения: x1 = 247.5, x2 = 0.0, x3 = 917.5
Max значение функции: 27042.5


In [7]:
#Реальный расход ресурсов
B_opt = np.array([x1 + 3*x2 + a*x3, 6*x1 + 5*x2 + 2*x3]) 
B_opt

array([3000., 3320.])

In [8]:
abs(B_map-B_opt)

array([0., 0.])

In [9]:
var_names = ['x1', 'x2', 'x3']          
#Задаем цены
profit = np.array([6*b + 12, 5*b + 22, c])  
#Задаем использование ресурсов
res_used = np.array([[1, 3, a], [6, 5, 2]]) 
#Задаем запасы
res_avail = np.array([3000, 3320])                    

In [10]:
n = len(profit)                                    
m = len(res_avail)                                 
model = Model()
x = model.addVars(n, name=var_names) # заводим переменные в модель
obj = model.setObjective(quicksum(profit[j] * x[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(res_used[i, j] * x[j] for j in range(n)) <= res_avail[i], name=var_names[i]))
model.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x91cab8a2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 3e+03]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.9000000e+31   3.375000e+30   7.900000e+01      0s
       2    2.7042500e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.704250000e+04


In [11]:
#Параметры модели:
print("X =", model.X, "\nFmax =", model.ObjVal, "\nОптимальный план двойственной задачи Pi → (y1,y2) =", model.Pi) 
print("x1 =", x[0].X)
print("Pi1 =", con[0].Pi)

X = [247.5, 0.0, 917.5] 
Fmax = 27042.5 
Оптимальный план двойственной задачи Pi → (y1,y2) = [5.625, 3.0625]
x1 = 247.5
Pi1 = 5.625


In [12]:
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])


    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
          x1        247.5           24      23.6667           69 
          x2            0           32         -inf      32.1875 
          x3        917.5           23      22.7692           72 


In [13]:
print('Fmax =', model.ObjVal)

Fmax = 27042.5


In [14]:
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])


    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
          x1        247.5            0            0         -inf        247.5          inf        247.5          inf 
          x2            0      -0.1875            0         -inf          440          inf            0          inf 
          x3        917.5            0            0         -inf        917.5          inf        917.5          inf 


In [15]:
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp'])


  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
          x1            <            0        5.625         3000      553.333         4980 
          x2            <            0       3.0625         3320         2000        18000 
